In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install lxml

In [2]:
from bs4 import BeautifulSoup
import lxml
import requests as r
import json
from datetime import date
from datetime import timedelta
import json

In [3]:
#get links of all categories
def getCateLinks():
    
    main_url = 'http://www.voanews.com/'
    res = r.get(main_url)
    res.encoding='unicode'
    soup = BeautifulSoup(res.text,'lxml')
    tags = soup.select('.block-secondary .inner .subitem a')
    
    urlList = []
    
    for tag in tags:
        if (tag['href'].lstrip('/').split('/')[0] != 'http:'):
            urlList.append('%s'%(main_url+tag['href'].lstrip('/')))
        else:
            urlList.append(tag['href'])
    
    return urlList

In [4]:
getCateLinks()

['http://www.voanews.com/z/599',
 'http://www.voanews.com/z/612',
 'http://www.voanews.com/z/600',
 'http://www.voanews.com/z/611',
 'http://www.voanews.com/z/598',
 'http://www.voanews.com/z/616',
 'http://www.voanews.com/z/621',
 'http://www.voanews.com/z/605',
 'http://www.voanews.com/z/607',
 'http://www.voanews.com/z/602',
 'http://projects.voanews.com/',
 'http://m.voanews.com/programindex.html',
 'http://m.voanews.com/p/6211.html',
 'http://blogs.voanews.com/',
 'http://editorials.voa.gov/']

In [5]:
#get all links of articles in each category
def getArticleList(cat_url, date):
    
    articleList = []
    
    res = r.get(cat_url + '/' + date + '/')
    res.encoding='unicode'
    soup = BeautifulSoup(res.text,'lxml')
    
    articles = soup.select('.media-block-wrap .row .content')
    
    for num in range(len(articles)):
        article_info = {}
        article_info['title'] = soup.select('.media-block-wrap .row .content .title')[num].text.strip()
        article_info['url'] = 'http://www.voanews.com' + soup.select('.media-block-wrap .row .content')[num].select('a')[0]['href']
        
        articleList.append(article_info)
    
    return articleList

In [6]:
getArticleList(getCateLinks()[0], '2017/04/16')

[{'title': "Trump's 'Magnetic' Pull Could Sway Election",
  'url': 'http://www.voanews.com/a/3812469.html'},
 {'title': 'With Trump Pick Aboard, Top US Court Tackles Religious Rights',
  'url': 'http://www.voanews.com/a/with-trump-pick-aboard-top-us-court-tackles-religious-rights/3812130.html'},
 {'title': 'Calendar Brings Western, Orthodox Christians Together for Easter',
  'url': 'http://www.voanews.com/a/thousands-come-to-hear-popes-easter-message/3812041.html'},
 {'title': 'US Urges Russia, Pakistan To End Support For Afghan Taliban',
  'url': 'http://www.voanews.com/a/top-us-security-official-in-afghanistan-for-talks/3812024.html'},
 {'title': 'Christians Around the World Celebrate Easter',
  'url': 'http://www.voanews.com/a/christian-around-the-world-celebrate-easter/3812016.html'},
 {'title': 'Fugitive Mexican ex-Gov. Javier Duarte Detained in Guatemala',
  'url': 'http://www.voanews.com/a/fugitive-former-mexican-governor-arrested-in-guatemala/3812008.html'},
 {'title': 'North K

In [7]:
inputList = getArticleList(getCateLinks()[0], '2017/04/16')

listIncludeContent = []    #output list

for num in range(len(inputList)):
    res = r.get(inputList[num]['url'])
    res.encoding='unicode'
    soup = BeautifulSoup(res.text,'lxml')
    contents = soup.select('.wsw p')
        
    paragraph_dict = {} 
    for numPara in range(len(contents)):
        paragraph_dict[numPara+1] = contents[numPara].text
    
    article_dict = {}    #store hole article details
    article_dict['title'] = inputList[num]['title']
    article_dict['date'] = soup.select('#content time')[0]['datetime'].split('T')[0]
    article_dict['source'] = 'VOA'
    article_dict['url'] = inputList[num]['url']
    article_dict['content'] = paragraph_dict
    listIncludeContent.append(article_dict)

IndexError: list index out of range

In [8]:
def addContent(articleList):
    
    listIncludeContent = []    #output list
    
    for num in range(len(articleList)):
        
        res = r.get(articleList[num]['url'])
        res.encoding='unicode'
        soup = BeautifulSoup(res.text,'lxml')
                
        contents = soup.select('.wsw p')
        
        paragraph_dict = {} 
        for numPara in range(len(contents)):
            paragraph_dict[numPara+1] = contents[numPara].text
        
        article_dict = {}    #store hole article details
        article_dict['title'] = articleList[num]['title']
        article_dict['date'] = soup.select('#content time')[0]['datetime'].split('T')[0]
        article_dict['source'] = 'VOA'
        article_dict['url'] = articleList[num]['url']
        article_dict['content'] = paragraph_dict
        
        listIncludeContent.append(article_dict)
            
    return listIncludeContent

In [9]:
def jsonOut(targetList):
    day = date.today().strftime('%Y%m%d')
    with open('{}_VOAoutput.json'.format(day), 'w') as f:
        json.dump(targetList, f)
        f.close()
    print('File {}_VOAoutput.json has been created!'.format(day))

In [10]:
jsonOut(addContent(test))

NameError: name 'test' is not defined